In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
import os
os.chdir('/home/jaentrouble/adipose_train2')
print(os.getcwd())

/home/jaentrouble/adipose_train2


In [3]:
!git pull
%autoreload

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0
Unpacking objects: 100% (5/5), 1.53 KiB | 781.00 KiB/s, done.
From https://github.com/jaentrouble/Adipos_tf_train2
   eb35c08..8c1fffd  master     -> origin/master
Updating eb35c08..8c1fffd
Fast-forward
 model_trainer.py  |  7 +++++++
 train_model.ipynb | 44 ++++++++++++++++++++++----------------------
 view_result.ipynb |  3 ++-
 3 files changed, 31 insertions(+), 23 deletions(-)


In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
with np.load('cell_mask_data.npz') as data:
    X = data['img']
    Y = data['mask']
print('X shape: ', X.shape, X.dtype)
print('Y shape: ', Y.shape, Y.dtype)

X shape:  (1533, 200, 200, 3) uint8
Y shape:  (1533, 200, 200) float32


In [11]:
X_test = X[1350:]
Y_test = Y[1350:]

In [7]:
import tensorflow as tf
from model_trainer import get_model
from adipose_models import *

In [9]:
model_f = conv4_b2_0
savedfile_name = 'conv4_b2_0'
target_epoch = 10

In [10]:
test_model = get_model(conv4_b2_0)
test_model.load_weights('savedmodels/'+savedfile_name+'/'+str(target_epoch))

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 32)      128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 200, 200, 32)      9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 200, 200, 32)     

In [12]:
test_model.evaluate(X_test, Y_test)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.